In [27]:
import os
import sys

import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.tools import optimize_for_inference_lib as optilib

from keras.models import load_model, Model
from keras import backend

# Inspect and extract graph
version = 3

model_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin/v' + str(version) + '/model.h5'

session = tf.Session()
backend.set_session(session)

model = load_model(model_path)
gd = session.graph.as_graph_def()

input_node_map = {}
for node in gd.node:
    if node.name not in input_node_map.keys():
        input_node_map[node.name] = node
    else:
        raise ValueError("Duplicate node names detected for ", node.name)

nodes_with_switch_op = [x for x in gd.node if x.op.lower().find('switch') != -1]
for n in nodes_with_switch_op:
    n.op = 'Identity'
    n.input.pop()

# input_node_map['dropout_1/cond/Switch_1']
# input_node_map['dropout_1/cond/pred_id']
        
# for node in gd.node:
#     for idx, i in enumerate(node.input):
#         input_clean = optilib.node_name_from_input(i)
#         if input_clean.endswith('/cond/Merge') and input_clean.split('/')[-3].startswith('dropout'):
#             identity = optilib.node_from_map(input_node_map, i).input[0]
#             print(identity)
#             if identity.split('/')[-1] == 'Switch_1':
#                 parent = optilib.node_from_map(input_node_map, optilib.node_from_map(input_node_map, identity).input[0])
#                 print(parent)
#                 pred_id = parent.input[1]
#                 if pred_id.split('/')[-1] == 'pred_id':            
#                     good = parent.input[0]
#                     node.input[idx] = good

nodes_with_learningphase = [x for x in gd.node if x.name.lower().find('keras_learning_phase') != -1]
print(nodes_with_learningphase)

for n in nodes_with_learningphase:
    n.op = 'Const'
    del n.attr['shape']
    
# for n in gd.node:
#     if 'dropout_1/cond/Switch_1' in [i for i in n.input]:
#         print(n)

# nodes_with_dropout = [(n.name, [i for i in n.input]) for n in gd.node if n.name.lower().find('dropout') != -1]
# print(nodes_with_dropout)

# nodes_with_learningphase = [x for x in gd.node if x.name.lower().find('keras_learning_phase') != -1]
# print(nodes_with_learningphase)

# output_graph_def = graph_util.convert_variables_to_constants(session, gd, ['main_output/Softmax'])

# WRITE TO GRAPH
# output_graph_def = gd

# tf.train.write_graph(output_graph_def, ".", "inference_graph.pb", as_text=False)


[name: "dropout_1/keras_learning_phase"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_BOOL
  }
}
attr {
  key: "shape"
  value {
    shape {
      unknown_rank: true
    }
  }
}
]
[name: "dropout_1/keras_learning_phase"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_BOOL
  }
}
]


In [23]:
def display_nodes(nodes):
    for i, node in enumerate(nodes):
        print('%d %s %s' % (i, node.name, node.op))
        [print(u'└─── %d ─ %s' % (i, n)) for i, n in enumerate(node.input)]